# Lab 3

In this lab, you will work with estimation methods based on likelihood.

In the presence of missing values, we maximize the observed likelihood :
$$\theta^{ML} \in \textrm{argmax}_\theta \: L_{{\mathrm{{obs}}}}(\theta;X_{\mathrm{obs}(M)}),$$

$$\mathrm{with} \: L_{{\mathrm{{obs}}}}(\theta;X_{\mathrm{obs}(M)})={\int} p(X;\theta) {dX_{\mathrm{mis}(M)}}.$$

This makes it possible to estimate the parameter $\theta$ of the data distribution $p(X; \theta)$, and potentially to predict missing values.


Exercise 1 will introduce you to the Expectation-Maximization algorithm in the case of a bivariate Gaussian distribution, to estimate parameters in the presence of missing values. In Exercise 2, you will see why and how these methods can also be used for imputation. In Exercise 3, you will work with low-rank methods, and in Exercise 4, with a method based on deep learning.

# Importing libraries

In [ ]:
### Classical libraries
import numpy as np
import pandas as pd

### Specific for missing values
from sklearn.impute import SimpleImputer

### Dataset
from sklearn.datasets import load_breast_cancer

### Data visualisation
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors

### Pytorch for exercice 4 (deep learning)
import torch
import torchvision
import torch.nn as nn
import torch.distributions as td
from torch import optim

# Exercise 1: Expectation-Maximization algorithm

Consider a bivariate Gaussian sample (*i.e.*, with \$d=2\$ variables),

$$
\left( X_{i.} \right)_{1\leq i\leq n}
=\left( X_{i0}, X_{i1} \right)_{1\leq i\leq n},
$$

of size \$n\$, with mean vector \$\mu\$ and covariance matrix \$\Sigma\$, where

$$
\mu = \begin{bmatrix}
  \mu_0 \\[6pt] \mu_1
\end{bmatrix}, \quad
\Sigma = \begin{bmatrix}
  \sigma_{0} & \sigma_{01} \\[6pt]
  \sigma_{01} & \sigma_{1}
\end{bmatrix}.
$$

Suppose further that there are missing values of type Missing Completely At Random (MCAR) in \$X\_{.1}\$: that is, the probability of missingness does not depend on the data values themselves, so the missing-data mechanism can be ignored in the statistical analysis.

In [ ]:
np.random.seed(0)

n = 100
d = 2
mu0 = 5.
mu1 = 1.
sig0 = 1.
sig1 = 1.
sig01 = 0.5

mean = np.array([mu0, mu1])
cov = np.array([
    [sig0, sig01],
    [sig01, sig1]
    ])

xfull = np.random.multivariate_normal(mean, cov, size=n)

In [ ]:
p = 0.4

xmiss = np.copy(xfull)
miss_id = (np.random.uniform(0, 1, size=n) < p)
xmiss[miss_id, 1] = np.nan

M = np.isnan(xmiss)

In [ ]:
ax = sns.scatterplot(x=xfull[:, 0], y=xfull[:, 1], hue=M[:, 1], palette=['#d1e5f0', '#2166ac'])
handles, labels  =  ax.get_legend_handles_labels()
ax.set_title('MCAR')
ax.set_xlabel(r'$X_{.0}$')
ax.set_ylabel(r'$X_{.1}$')
ax.legend(handles, ['Observed', 'Missing'], loc='lower right', fontsize='13')
;

## Question 1 : maximum likelihood estimator

The observed log-likelihood to be maximized is
$$\ell_{\textrm{obs}}(\theta;X_{.0},X_{\mathrm{obs}(M_{.1})1})=\sum_{i=1}^n \int \log p(X_{i.};\theta) dX_{\mathrm{mis}(M_{i1})1},$$
where $\theta=(\mu,\Sigma)$. By default, we have $X_{i1}=X_{\mathrm{mis}(M_{i1})1}$ if $M_{i1}=1$, that is $X_{i1}$ is missing. Similarly, we have $X_{i1}=X_{\mathrm{obs}(M_{i1})1}$ if $M_{i1}=0$, that is when $X_{i1}$ is observed.
In the following, you can consider, without loss of generality, that the first $r$ observations are observed and the remaining $n-r$ are missing, that is, $M_{i1}=0$ for $i=1,\dots,r$ and $M_{i1}=1$ for $i=r+1,\dots,n$.

In Question 1a, you will show that its expression is, up to some constants:
$$\ell(\mu,\Sigma;X_{.0},X_{\mathrm{obs}(M_{.1})1})=-\frac{n}{2}\log(\sigma_{0}^2)-\frac{1}{2}\sum_{i=1}^{n}\frac{(X_{i0}-\mu_0)^2}{\sigma_{0}^2}
-\frac{r}{2}\log\left(\sigma_{1}-\frac{\sigma_{01}^2}{\sigma_{0}}\right)^2 \\
-\frac{1}{2}\sum_{i=1}^{r}\frac{(X_{i1}-\mu_1-\frac{\sigma_{01}}{\sigma_{0}}(X_{i0}-\mu_0))^2}{\left(\sigma_{11}-\frac{\sigma_{01}^2}{\sigma_{0}}\right)^2}$$
and in Question 1b, you will provide the expression of the maximum likelihood estimator. Question 1c involves the implementation.

### Question 1a

Here are the first steps to obtain the observed likelihood.

\begin{align}
\ell_{\textrm{obs}}(\theta;X_{.0},X_{\mathrm{obs}(M_{.1})1})&=\sum_{i=1}^n \int \log p(X_{i.};\theta) dX_{\mathrm{mis}(M_{.1})1} \\
&=\sum_{i=1}^n \int \log p(X_{i0})p(X_{i0}|X_{i1};\theta) dX_{\mathrm{mis}(M_{.1})1} \\
&= \sum_{i=1}^n \log p(X_{i0}) + \sum_{i=1}^n \int \log p(X_{i0}|X_{i1};\theta) dX_{\mathrm{mis}(M_{.1})1}
\end{align}

Detail the two terms, using the classical formulas for a Gaussian vector recalled below for the second term :

$X_{i1}|X_{i0} \sim N(\mathbb{E}[X_{i1}|X_{i0}],\mathrm{Var}(X_{i1}|X_{i0}))$ avec
\begin{align*}
\mathbb{E}[X_{i1}|X_{i0}]&=\mu_1+\frac{\sigma_{01}}{\sigma_{0}}(X_{i0}-\mu_0) \\
\mathrm{Var}(X_{i1}|X_{i0})&=\sigma_{1}-\frac{\sigma_{01}^2}{\sigma_{0}}
\end{align*}

### Solution

For the first term, we have $X_{.0}\sim N(\mu_0,\sigma_{0})$, then
$\sum_{i=1}^n \log p(X_{i0};\theta)= -\sum_{i=1}^n \frac{(X_{i0}-\mu_0)^2}{2\sigma_{0}^2}-\frac{n}{2}\log\sigma_{0}^2$.

For the second term, there are two cases. If $X_{i1}$ is missing, the integral is equal to 1 (integral of a density on the whole space).
If $X_{i1}$ is observed (that is for $X_{i1}$ with $i=1,\dots,r$), the term $p(X_{i1}|X_{i1};\theta)$ can be taken out of the integral, and standard formulas are then used to obtain : $-\frac{r}{2}\log\left(\sigma_{1}-\frac{\sigma_{01}^2}{\sigma_{0}}\right)^2
-\frac{1}{2}\sum_{i=1}^{r}\frac{(X_{i1}-\mu_1-\frac{\sigma_{01}}{\sigma_{0}}(X_{i0}-\mu_0))^2}{\left(\sigma_{1}-\frac{\sigma_{01}^2}{\sigma_{0}}\right)^2}$.



### Question 1b

The maximum likelihood estimator is
$\theta^{ML} \in \textrm{argmin}_\theta \: \ell_{\textrm{obs}}(\theta;X_{.0},X_{\mathrm{obs}(M_{.1})1})$. Give its expression to estimate the mean, assuming $\Sigma$ is known to simplify the calculations.

### Solution

With $\Sigma$ fixed (known), differentiating the observed likelihood with respect to $\mu$ yields the following expressions:

\begin{align*}
    \nabla_{\mu_0} \ell(\mu,\Sigma;X_{.0},X_{.1})&=\sum_{i=1}^{n}\frac{X_{i0}-\mu_0}{\sigma_{0}^2}
 \\
 \nabla_{\mu_1}\ell(\mu,\Sigma;X_{.0},X_{.1})&=\sum_{i=1}^{r}\frac{X_{i1}-\mu_1-\frac{\sigma_{01}}{\sigma_{0}}(X_{i0}-\mu_0)}{\left(\sigma_{1}-\frac{\sigma_{01}^2}{\sigma_{0}}\right)^2}
\end{align*}

\begin{align*}
\nabla_{\mu_0} \ell(\mu,\Sigma;X_{.0},X_{.1})=0 &\Longleftrightarrow {\mu}^{ML}_0 = \frac{1}{n}\sum_{i=1}^n X_{i0} \\
\nabla_{\mu_1} \ell(\mu,\Sigma;X_{.0},X_{.1})=0 &\Longleftrightarrow {\mu}^{ML}_1 = \frac{1}{r}\sum_{i=1}^r X_{i1} + \frac{\sigma_{01}}{\sigma_{0}}\left({\mu}_0^{ML}-\frac{1}{r} \sum_{i=1}^r X_{i0}\right),
\end{align*}
where $\mu_0$ is replaced by its estimator ${\mu}^{ML}_0$ in the expression of ${\mu}^{ML}_1$ (plug-in method). The proof of concavity for the likelihood $\ell$ is not provided.

### Question 1c

Implement the maximum likelihood estimator for the mean based on the expression derived in question 1b.

### Solution

In [ ]:
mu0_ML = np.mean(xmiss[:, 0])
mu1_ML = np.mean(xmiss[~miss_id, 1]) + sig01 / sig0 * (mu0_ML - np.mean(xmiss[~miss_id, 0]))

In [ ]:
print("Estimator of mu0 :", mu0_ML)
print("Estimator of mu1 :", mu1_ML)

## Question 2 : E-step

As seen in the video from Module 3, the parameter $\theta$ can also be estimated iteratively using the EM algorithm.

There is an initialization step where one obtains $\theta^{(0)}$, followed by two steps repeated until convergence:
* the E-step, where the following conditional expectation is computed (at iteration $t$):
$$Q(\theta;{\theta}^{(t)})=\mathbb{E}[\quad \ell(\theta;X)\quad\big|X_{.0},X_{\mathrm{obs}(M_{.1})1},{\theta}^{(t)}],$$
with $\ell(\theta;X)=\sum_{i=1}^n\log p(X;\theta)$ being the complete-data log-likelihood.

* the M-step, where the conditional expectation is maximized to update the parameters
$${\theta}^{(t+1)}\in\textrm{argmax}_\theta \: Q(\theta;{\theta}^{(t)})$$

You will detail the E-step.



The complete-data log-likelihood is given below:

\begin{align*}
\ell(\theta;X)&=-\frac{n}{2}\log(\mathrm{det}(\Sigma))-\frac{1}{2}\sum_{i=1}^n\begin{pmatrix} X_{i0}-\mu_0 & X_{i1}-\mu_1
\end{pmatrix}\Sigma^{-1}\begin{pmatrix} X_{i0}-\mu_0 & X_{i1}-\mu_1
\end{pmatrix}^T \\
&= \frac{n}{2}\log(\mathrm{det}(\Sigma))-\frac{1}{2}\sum_{i=1}^n (X_{i0}-\mu_0)^2\tilde{\sigma}_{0} + 2(X_{i0}-\mu_0)(X_{i1}-\mu_1)(\tilde{\sigma}_{01})^2 + (X_{i1}-\mu_1)^2\tilde{\sigma}_{1},
\end{align*}
avec $\Sigma^{-1}=\begin{pmatrix} \tilde{\sigma}_{0} & \tilde{\sigma}_{01} \\
\tilde{\sigma}_{01} & \tilde{\sigma}_{1}  
\end{pmatrix}$.

$\ell(\theta;X)$ is linear in the following quantities: $\sum_{i=1}^n X_{i0}$, $\sum_{i=1}^n X_{i0}^2$, $\sum_{i=1}^n X_{i1}$, $\sum_{i=1}^n X_{i1}^2$ and $\sum_{i=1}^n X_{i0}X_{i1}$.
To obtain the expression of the conditional expectation, it is therefore sufficient to compute the following quantities:
\begin{align*}
s_j&=\mathbb{E}\left[ \sum_{i=1}^n X_{ij}|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)}\right], j=0,1 \\
s_{jj}&=\mathbb{E}\left[ \sum_{i=1}^n X_{ij}^2|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)}\right], j=0,1 \\
s_{jk}&=\mathbb{E}\left[ \sum_{i=1}^n X_{ij}X_{ik}|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)}\right], j=0, k=1
\end{align*}


### Question 2a

Calculate $s_0$ and $s_{00}$.

### Solution

The expressions for $s_0$ and $s_{00}$ are the easiest to obtain. Indeed, since we are conditioning on $X_{.0}$, we directly get:
$$s_0 = \sum_{i=1}^n X_{i0}, \quad  s_{00} = \sum_{i=1}^n X_{i0}^2.$$

### Question 2b

Calculate $s_1$. The technique is the same as the one used for computing the second term in question 1a.

### Solution

\begin{align*}
        \mathbb{E}\left[ X_{i1}|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)}\right]&=  \int X_{i1} p(X_{\textrm{mis}(M_{i1})1}|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)})dX_{\textrm{mis}(M_{i1})1} \\
        &=\begin{cases}
		X_{\textrm{obs}(M_{i1})1}  &\textrm{ if } X_{i1} \textrm{ is observed} \\
		\int X_{\textrm{mis}(M_{i1})1} p(X_{\textrm{mis}(M_{i1})1}|X_{i0},{\theta}^{(t)})dX_{\textrm{mis}(M_{i1})1} &\textrm{ otherwise.}
	\end{cases}
\end{align*}
In the first case, if $X_{i1}$ is observed, then $X_{i1} = X_{\textrm{obs}(M_{i1})1}$ and can be taken out of the integral, which equals 1. In the second case, it is simply the expectation of the conditional distribution of $X_{i1}$ given $X_{i0}$. To compute this, one can use the standard formulas for Gaussian vectors (as recalled in question 1a), and obtain:
\begin{align*}
        \mathbb{E}\left[ X_{i1}|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)}\right]&= \begin{cases}
		X_{i1} &\textrm{ if } X_{i1} \textrm{ is observed} \\
		\mu_1^{(t)} + \frac{\sigma_{01}^{(t)}}{\sigma_{0}^{(t)}}( X_{i0}-\mu_0^{(t)}) &\textrm{ otherwise.}
	\end{cases}
\end{align*}

For the terms $s_{11}$ and $s_{01}$, exactly the same strategy as in question 2b is used to obtain:

\begin{align}
s_{11}&=\mathbb{E}\left[ \sum_{i=1}^nX_{i1}^2|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)}\right]=\sum_{i=1}^{r} X_{i1}^2 + \sum_{i=r+1}^n \left(\left(\mu_1^{(t)} + \frac{\sigma^{(t)}_{01}}{\sigma^{(t)}_{0}}( X_{i0}-\mu^{(t)}_0)\right)^2+\sigma^{(t)}_{1}-\frac{(\sigma^{(t)}_{01})^2}{\sigma^{(t)}_{0}}\right) \\
s_{01}&=\mathbb{E}\left[ \sum_{i=1}^n X_{i0}X_{i1}|X_{i0},X_{\mathrm{obs}(M_{i1})1},{\theta}^{(t)}\right]=\sum_{i=1}^{r} X_{i0}X_{i1} + \sum_{i=r+1}^n X_{i0}\left(\mu^{(t)}_1 +\frac{\sigma^{(t)}_{01}}{\sigma^{(t)}_{0}}\left( X_{i0}-\mu^{(t)}_0\right)\right)
\end{align}

## Question 3: M-step

Without detailing the calculations, how can the likelihood be maximized in the M-step?

### Solution

The classical maximum likelihood estimator in the Gaussian case is recovered by replacing the quantities involving missing values with the conditional expectations computed in question 2 (see for example Section 2.3.4 of
[the book of Bishop, 2006](https://www.microsoft.com/en-us/research/wp-content/uploads/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf)).

\begin{align*}
    \mu_0^{(t+1)}&=\frac{s_0}{n} \\
    \mu_1^{(t+1)}&=\frac{s_1}{n} \\
    \sigma_{0}^{(t+1)}&=\frac{s_{00}}{n}-(\mu_0^{(t+1)})^2 \\
    \sigma_{1}^{(t+1)}&=\frac{s_{11}}{n}-(\mu_1^{(t+1)})^2 \\
    \sigma_{01}^{(t+1)}&=\frac{s_{01}}{n}-(\mu_0^{(t+1)}\mu_1^{(t+1)})
\end{align*}

## Question 4: Practical implementation

### Question 4a

First, an initialization for $\theta^{(0)}$ must be found. The algorithm will be initialized with empirical quantities (mean and covariance matrix) computed from the complete cases. This is not always a viable solution (for example, when there are too many missing values). Another approach can be to impute the missing values using a computationally inexpensive method and then compute the empirical quantities for $\theta^{(0)}$.

Write the code that performs the initialization. What do you observe?

### Solution

The results for the covariance matrix $\Sigma$ are biased.

In [ ]:
mu_init = np.nanmean(xmiss,axis=0)
mu_init

In [ ]:
Sigma_init = np.array(pd.DataFrame(xmiss).cov())
Sigma_init

### Question 4b

Write two functions for the E-step and the M-step of the algorithm.

In [ ]:
def Estep(xmiss, mu, Sigma, miss_id):

    ### TO COMPLETE ###

    return {
        's0': np.sum(s0),
        's1': np.sum(s1),
        's00': np.sum(s00),
        's11': np.sum(s11),
        's01': np.sum(s01)
    }

In [ ]:
def Mstep(xmiss, s0, s1, s00, s11, s01):

    ### TO COMPLETE ###

    return {'mu': mu, 'Sigma': Sigma}

### Solution

In [ ]:
def Estep(xmiss, mu, Sigma, miss_id):

    n = xmiss.shape[0]

    # For the variable X0
    s0 = xmiss[:, 0]
    s00 = xmiss[:, 0] ** 2

    s1 = np.zeros(n)
    s11 = np.zeros(n)

    # For observed values of X1
    s1[~miss_id] = xmiss[~miss_id, 1]
    s11[~miss_id] = xmiss[~miss_id, 1] ** 2

    # For missing values of X1
    s1[miss_id] = mu[1] + (Sigma[0,1] / Sigma[0,0]) * (xmiss[miss_id, 0] - mu[0])
    s11[miss_id] = (
        s1[miss_id] ** 2 +
        Sigma[1,1] - (Sigma[0,1] ** 2) / Sigma[0,0]
    )

    s01 = s0 * s1

    return {
        's0': np.sum(s0),
        's1': np.sum(s1),
        's00': np.sum(s00),
        's11': np.sum(s11),
        's01': np.sum(s01)
    }

In [ ]:
def Mstep(xmiss, s0, s1, s00, s11, s01):

    n = xmiss.shape[0]

    mu0 = s0 / n
    mu1 = s1 / n

    sig0 = s00 / n - mu0 ** 2
    sig1 = s11 / n - mu1 ** 2
    sig01 = s01 / n - mu0 * mu1

    mu = np.array([mu0, mu1])
    Sigma = np.array([
        [sig0, sig01],
        [sig01, sig1]
    ])

    return {'mu': mu, 'Sigma': Sigma}

### Question 4c

Run the algorithm for 50 iterations.

### Solution

In [ ]:
mu_EM = mu_init
Sigma_EM = Sigma_init

for i in range(50):

  # E-step
  res_Estep = Estep(xmiss, mu_EM, Sigma_EM, miss_id)
  s0 = res_Estep["s0"]
  s1 = res_Estep["s1"]
  s00 = res_Estep["s00"]
  s11 = res_Estep["s11"]
  s01 = res_Estep["s01"]

  # M-step
  res_Mstep = Mstep(xmiss, s0, s1, s00, s11, s01)
  mu_EM = res_Mstep["mu"]
  Sigma_EM = res_Mstep["Sigma"]


In [ ]:
mu_EM

In [ ]:
Sigma_EM

# Exercise 2: estimate in order to impute

In this short exercise, you will impute the missing values based on the estimation of the parameter $\theta$ of the data distribution $p(X)$.

### Question 1

In the case of Exercise 1 (bivariate Gaussian), propose a strategy to impute, that is, to draw a sample following the distribution $p(X_{i1}|X_{i0})$.

### Solution

In the bivariate Gaussian case, this distribution is explicit. The classic formulas for a Gaussian vector, recalled in question 1a, can be used again.

We have:
$X_{i1}|X_{i0} \sim N(\mathbb{E}[X_{i1}|X_{i0}],\mathrm{Var}(X_{i1}|X_{i0}))$ with
\begin{align*}
\mathbb{E}[X_{i1}|X_{i0}]&=\mu_1+\frac{\sigma_{01}}{\sigma_{0}}(X_{i0}-\mu_0) \\
\mathrm{Var}(X_{i1}|X_{i0})&=\sigma_{1}-\frac{\sigma_{01}^2}{\sigma_{0}}
\end{align*}

## Question 2

Implement the chosen strategy.

### Solution

In [ ]:
def imputation_EM(mu, Sigma, xmiss, miss_id):

  ximp = np.copy(xmiss)

  for i in range(xmiss.shape[0]):
    if miss_id[i]:
      mean = mu[1] + (Sigma[0, 1] / Sigma[0, 0]) * (xmiss[i, 0] - mu[0])
      cov = Sigma[1, 1] - (Sigma[0, 1] ** 2) / Sigma[0,0]
      ximp[i, 1] = np.random.normal(mean, cov, size=1)

  return ximp

In [ ]:
ximp_EM = imputation_EM(mu_EM, Sigma_EM, xmiss, miss_id)

In [ ]:
ax = sns.scatterplot(x=ximp_EM[:, 0], y=ximp_EM[:, 1], hue=M[:, 1], palette=['#d1e5f0', '#2166ac'])
handles, labels  =  ax.get_legend_handles_labels()
ax.set_title('MCAR values imputed with the EM algorithm')
ax.set_xlabel(r'$X_{.0}$')
ax.set_ylabel(r'$X_{.1}$')
ax.legend(handles, ['Observed', 'Imputed'], loc='lower right', fontsize='13')
;

# Exercise 3: low-rank methods

In this exercise, you will implement the algorithm presented in the module video, which predicts missing values using iterative Principal Component Analysis (PCA). In question 2, more details will be provided on one of its variants, which better handles noise in the data, called `missMDA`: this method is implemented in a widely used R package. In question 3, you will implement another of its variants yourselves, the `softImpute` algorithm.

In low-rank methods, it is assumed that the data matrix can be decomposed into a low-rank matrix plus Gaussian noise.
$$X = \Theta + \epsilon,
$$
with $\epsilon\sim N(0_d,\sigma^2I_{d\times d})$.

To predict the missing values in $X$, the parameter $\Theta$ will be estimated using the iterative PCA algorithm (presented in the video). The algorithm takes as input $r$, the number of dimensions to retain for PCA, as well as the incomplete dataset $X^\textrm{NA}$.

- There is a step of **naive initialization** for imputation (for example, by the mean or even imputing with 0). This yields a complete dataset $X^{(0)}$.

Two steps are performed iteratively:

- the **estimation step**: $\theta$ is estimated using the singular value decomposition, keeping only the first $r$ dimensions.
$$\Theta^{(t)}=\text{SVD}_{\textbf{r}}({X}^{(t)})=U_{{\textbf{r}}}D_{{\textbf{r}}}V_{{\textbf{r}}}^t,$$

- the **imputation step**: the missing values are predicted by the value of $\Theta^{(t)}$.

$$X^{(t+1)}=X \odot (\mathbf{1}_{n\times d}-{M}) + \: \Theta^{(t)} \odot M.$$


In this exercise, the same complete real dataset *Breast Cancer Wisconsin* as in the previous modules (Exercise 4 of Lab 1 and Exercise 3 of Lab 2) will be considered. We introduce 30% missing values of the MCAR type.

In [ ]:
data = load_breast_cancer()
xfull = data['data'] # covariates, without missing values
diagnosis = data['target']   # target variable to predict, when the learning task is prediction
features_names = data['feature_names']

In [ ]:
pd.DataFrame(xfull, columns=features_names).head()

In [ ]:
n, d = xfull.shape

In [ ]:
np.random.seed(123)

p = 0.3
xmiss = np.copy(xfull)
for j in range(d):
  miss_id = np.random.uniform(0, 1, size=n) < p
  xmiss[miss_id, j] = np.nan
mask = np.isnan(xmiss)

The focus will be on the imputation task, and the mean squared errors (MSE) will be calculated to evaluate the methods (score introduced in Lab 2).

In [ ]:
def mse(x_imp, x_true):
  n = len(x_true)
  return (1 / n) * np.sum((x_imp - x_true) ** 2)

## Question 1: iterative PCA

### Question 1a

Propose a naive imputation by the mean and another by 0 in functions called `init_mean_imputation` and `init_zero_imputation`.

### Solution

In [ ]:
def init_mean_imputation(xmiss, mask):
    mean_imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
    ximp = mean_imputer.fit_transform(xmiss)
    return ximp

In [ ]:
def init_zero_imputation(xmiss, mask):
    ximp = xmiss.copy()
    ximp[mask] = 0
    return ximp

In [ ]:
pd.DataFrame(init_mean_imputation(xmiss, mask)).head()

In [ ]:
pd.DataFrame(init_zero_imputation(xmiss, mask)).head()

### Question 1b

Implement the algorithm of iterative PCA in the function `iterative_ACP` which takes as arguments:
* `xmiss`: the incomplete dataset,
* `r`: the number of dimensions to keep for PCA,
* `maxit`: the maximum number of iterations of the algorithm. A convergence criterion will be defined in question 3.

The mean imputation will be used as initialization.

For the singular value decomposition, you can use the function `linalg.svd` of `numpy` (available documentation [here](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html)).

In [ ]:
def iterative_ACP(xmiss, r, maxit):

    ximp = ...

    return ximp

### Solution

In [ ]:
def iterative_ACP(xmiss, r, maxit):

    mask = np.isnan(xmiss)

    ximp = init_mean_imputation(xmiss, mask)

    for i in range(maxit):
        U, d, V = np.linalg.svd(ximp, compute_uv = True, full_matrices=False)
        d_r = d[:r]
        U_r = U[:, :r]
        V_r = V[:r, :]
        D_r = np.diag(d_r)
        svd_r = np.dot(U_r, np.dot(D_r, V_r))
        ximp[mask] = svd_r[mask]

    return ximp

In [ ]:
pd.DataFrame(iterative_ACP(xmiss, 5, 50)).head()

### Question 1c

Calculate the MSE, and compare it with the one obtained using mean imputation.

### Solution

In [ ]:
mean_imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
ximp_mean = mean_imputer.fit_transform(xmiss)
mse_mean = mse(ximp_mean,xfull)
print("MSE imputation par la moyenne:", mse_mean)

In [ ]:
ximp_ACP = iterative_ACP(xmiss, 5, 1000)
mse_ACP = mse(ximp_ACP, xfull)
print("MSE ACP itérative:", mse_ACP)

A slightly lower MSE is obtained compared to mean imputation. It is worth questioning whether the number of dimensions $r$ to keep in the PCA was well chosen here.

### Question 1d

To choose the optimal number $r$ of dimensions to keep for PCA, one solution is to search over a grid of possible values and select the $r_{\mathrm{opt}}$ that minimizes the MSE on this grid.

Implement this strategy using the following function `additional_na`. It allows adding MCAR-type missing values such that there is at least one observed value remaining in each row.

In [ ]:
def additional_na(xmiss, mask):

    xmiss_addna = np.copy(xmiss)

    for i in range(xmiss.shape[0]):
        idx_obs = np.argwhere(mask[i, :] == 0).reshape((-1))
        var_obs = np.random.choice(idx_obs) # choice of the observed variable
        idx_candidates = idx_obs[idx_obs != var_obs]
        var_miss = np.random.uniform(0, 1, size=len(idx_candidates)) < 0.5
        missing_idx = idx_candidates[var_miss]
        xmiss_addna[i, missing_idx] = np.nan
    mask_addna = np.isnan(xmiss_addna)

    return(xmiss_addna, mask_addna)

### Solution

In [ ]:
def choose_rank(xmiss, grid_r):

    mask = np.isnan(xmiss)

    list_error = []
    for r in grid_r:
        xmiss_addna, mask_addna = additional_na(xmiss, mask)
        ximp_ACP = iterative_ACP(xmiss_addna, r, 1000)

        list_error.append(np.sqrt(np.nanmean((ximp_ACP.flatten() - xmiss.flatten())**2)))

    return list_error

In [ ]:
grid_r = list(range(1, d))
res_rank = choose_rank(xmiss, grid_r)

In [ ]:
r_opt = grid_r[np.argmin(res_rank)]
ximp_ACP = iterative_ACP(xmiss, r_opt, 1000)
mse_ACP = mse(ximp_ACP, xfull)
print("MSE ACP itérative:", mse_ACP)
print("r optimal:", r_opt)

The chosen optimal $r$ is 1. Thus, a projection of the data onto a single dimension is used to predict the missing values. This results in a significant loss of information, and one possible approach would be to better handle the noise contained in the data.

## Question 2 : regularized iterative PCA

In the algorithm of iterative PCA, the missing values are replaced by : $$\sum_{k=1}^{r}\sigma_k(X)u_{ik}v_{jk},$$
for $i=1,\dots,n$ and $j=1,\dots,p$.

To regularize, the `missMDA` algorithm proposes to replace the missing values by: $$\sum_{k=1}^{r}\left(\sigma_k(X)-\frac{{\sigma}^2}{\sigma_k(X)}\right)u_{ik}v_{jk}.$$

This method is implemented in `R` in the `missMDA` package. Other examples of applications are available in this [link](http://factominer.free.fr/missMDA/index.html) and the research paper can be found [here](https://www.jstatsoft.org/article/view/v070i01).

Make a list of the hyperparameters to choose in the `missMDA` method.

### Solution

The hyperparameters are $r$ (the number of dimensions to keep in the PCA) and $\sigma^2$ (the variance of the assumed Gaussian noise).

To estimate $r$, a method similar to the one chosen in question 1 can be used. To estimate $\sigma^2$, it has been proposed to use the sum of squared residuals divided by $nd - \#\textrm{param}$, where $n$ and $d$ are the dimensions of the data matrix, and $\#\textrm{param} = nr + pr - r^2$ is the number of parameters to estimate (here, the number of parameters in a singular value decomposition). More details are available [here](https://arxiv.org/pdf/1602.01206).

Note that there is a function `estim_ncpPCA` in the `missMDA` package which proposes a cross-validation procedure to estimate the number of dimensions to keep in the PCA.

## Question 3: soft-thresholded iterative PCA

Another method that allows handling noise in the data is to use soft-thresholding with the `softImpute` algorithm. Like `missMDA`, this algorithm better accounts for the noise present in the data compared to classical iterative PCA, by replacing missing values with:
$$\sum_{k=1}^{d}\max((\sigma_k(X)-\lambda),0)u_{ik}v_{jk},$$
with $\lambda>0$.

The estimation of $\theta$ is regularized in the sense that the optimization problem becomes:

$$\theta \in \textrm{argmin}_\theta \| (X - \Theta) \odot (\mathbf{1}_{n\times d}-M)\| + \lambda \|\Theta\|_\star,$$

with $\|.\|$ the nuclear norm.

The paper associated with this method is available [here](https://arxiv.org/pdf/1410.2596). The algorithm is implemented in `R` in the `softImpute` package.

### Question 3a

What are the hyperparameters?

### Solution

The hyperparameter is $\lambda$.

### Question 3b

Implement this method by building on the following base code. Use the zero imputation coded in question 1a.

In [ ]:
def softimpute(xmiss, lamb, maxit = 1000):

    mask = np.isnan(xmiss)

    ximp = ...

    for i in range(maxit):
        U, d, V = np.linalg.svd(ximp, compute_uv = True, full_matrices=False)
        d_thresh = ...
        r = ...
        d_thresh = d_thresh[:r]
        U_thresh = U[:, :r]
        V_thresh = V[:r, :]
        D_thresh = np.diag(d_r)
        svd_r = np.dot(U_thresh, np.dot(D_thresh, V_thresh))
        ximp[mask] = svd_r[mask]

    return ximp

### Solution

In [ ]:
def softimpute(xmiss, lamb, maxit = 1000):

    mask = np.isnan(xmiss)

    ximp = init_mean_imputation(xmiss, mask)

    for i in range(maxit):
        U, d, V = np.linalg.svd(ximp, compute_uv = True, full_matrices=False)
        d_thresh = np.maximum(d - lamb, 0)
        r = (d_thresh > 0).sum()
        d_thresh = d_thresh[:r]
        U_thresh = U[:, :r]
        V_thresh = V[:r, :]
        D_thresh = np.diag(d_thresh)
        svd_r = np.dot(U_thresh, np.dot(D_thresh, V_thresh))
        ximp[mask] = svd_r[mask]

    return ximp

In [ ]:
ximp_soft = softimpute(xmiss, 100, maxit = 1000)

In [ ]:
mse(ximp_soft,xfull)

### Question 3c

To avoid unnecessary iterations, the convergence criterion implemented below can be used, which compares the difference between two successive iterates. If the difference is smaller than a threshold `conv_thresh` (set to $10^{-5}$ by default), the algorithm is considered to have converged.

Incorporate this convergence criterion into the function `softimpute`.

In [ ]:
def converged(x_t, x_tplus1, mask, conv_thresh):

    x_t_na = x_t[mask]
    x_tplus1_na = x_tplus1[mask]
    rmse = np.sqrt(np.sum((x_t_na - x_tplus1_na) ** 2))
    denom = np.sqrt((x_t_na ** 2).sum())

    return (rmse / denom) < conv_thresh

### Solution

In [ ]:
def softimpute(xmiss, lamb, maxit = 1000, conv_thresh = 1e-5):

    mask = np.isnan(xmiss)

    ximp = init_mean_imputation(xmiss, mask)

    for i in range(maxit):
        U, d, V = np.linalg.svd(ximp, compute_uv = True, full_matrices=False)
        d_thresh = np.maximum(d - lamb, 0)
        r = (d_thresh > 0).sum()
        d_thresh = d_thresh[:r]
        U_thresh = U[:, :r]
        V_thresh = V[:r, :]
        D_thresh = np.diag(d_thresh)
        svd_r = np.dot(U_thresh, np.dot(D_thresh, V_thresh))
        if converged(ximp, svd_r, mask, conv_thresh):
            break
        ximp[mask] = svd_r[mask]

    return ximp

### Question 3d

Use the function `choose_lambda` below, which explores a grid of values for $\lambda$ and uses the function additional_na defined in question 1d.

`grid_len` sets the size of the grid of values.
The grid of values is defined inside the `choose_lambda` function by computing the maximum singular value of the dataset imputed with zeros.

In [ ]:
def choose_lambda(xmiss, grid_len = 15, maxit = 1000, conv_thresh = 1e-5):

    mask = np.isnan(xmiss)
    ximp_0 = init_zero_imputation(xmiss, mask)

    # generate grid for lambda values
    d = np.linalg.svd(ximp_0, compute_uv=False, full_matrices=False) # svd on imputed dataset with 0
    lambda_max = np.max(d)
    lambda_min = 0.001 * lambda_max
    grid_lambda = np.exp(np.linspace(np.log(lambda_min), np.log(lambda_max), grid_len).tolist())

    cv_error = []
    for lamb in grid_lambda:
        xmiss_addna, mask_maskna = additional_na(xmiss, mask)
        ximp_soft = softimpute(xmiss_addna, lamb, maxit, conv_thresh)
        cv_error.append(np.sqrt(np.nanmean((ximp_soft.flatten() - xmiss.flatten()) ** 2)))

    return cv_error, grid_lambda


### Solution

In [ ]:
res_lambda, grid_lambda = choose_lambda(xmiss, grid_len = 15)
lambda_opt = grid_lambda[np.argmin(res_lambda)]
ximp_soft = softimpute(xmiss, lambda_opt)
mse_soft = mse(ximp_soft, xfull)
print("MSE softImpute:", mse_soft)

# Exercise 4: method using deep learning

In this exercise, you will predict the missing values in the *Breast Cancer Wisconsin* dataset (from exercise 3) using the `MIWAE` method. The research paper proposing this method is available [here](https://arxiv.org/pdf/1812.02633). Notebooks are available in the Github repository [here](https://github.com/pamattei/miwae/tree/master). The code is taken from [this notebook](https://github.com/pamattei/miwae/blob/master/Tensorflow%202%20notebooks/MIWAE_UCI_single_multiple-imputation.ipynb).

There are no implementation questions in this exercise; the goal is to better understand the method. The computations can be done on CPU or GPU but will be faster if a GPU is available.



`MIWAE` uses a deep latent variable model. Let $Z \in \mathbb{R}^{d_{\textrm{lat}}}$ be the latent variables. The idea is to encode the information contained in the data into the latent space (of smaller dimension, $d_{\textrm{lat}} < d$). The information is then decoded back to the data space with a distribution $p_\theta(X|Z)$ parameterized by $f_\theta(Z)$, which is trained by a neural network.

`MIWAE` proposes to estimate the parameter $\theta$ by computing a lower bound $L_K$ of the observed likelihood, such that $L_K \leq L_{\mathrm{obs}}$. This bound draws samples for the latent variables according to a distribution $p_\gamma(Z|X_{\mathrm{obs}(M)})$, which encodes the data into the latent space, and which is parameterized by $g_\gamma(X_{\mathrm{obs}(M)})$, trained by another neural network.
$$L_K(\theta,\gamma)=\sum_{i=1}^n \mathbb{E}_{Z_{i1},\dots,Z_{iK}\sim p_\gamma(Z|X_{\mathrm{obs}(M)})}\left[\log \frac{1}{K}\sum_{k=1}^K \frac{p_\theta(X_{\mathrm{obs}(M)}|Z_{ik})p(Z_{ik})}{{p_\gamma(Z_{ik}|X_{\mathrm{obs}(M)})}}\right].$$

Finally, the generative model is as follows:
$$
\left\{
    \begin{array}{ll}
        Z\sim p_\gamma(Z|X_{\mathrm{obs}(M)};{g_\gamma(X_{\mathrm{obs}(M)})}) &  \textrm{(encoder)} \\
        X_{\mathrm{obs}(M)} \sim p_\theta(X_{\mathrm{obs}(M)}|Z;{f_\theta(Z)}) & \textrm{(decoder)}
    \end{array}
\right.
$$

with $g_\gamma(X_{\mathrm{obs}(M)})$ and $f_\theta(Z)$ trained by neural networks.

## Question 1: hyperparameters and model

What are the hyperparameters? What choices need to be made for this imputation method?

### Solution

The hyperparameters are $d_{\textrm{lat}}$ (dimension of the latent space) and $K$ (number of samples for the importance sampling).

The distributions $p(Z)$, $p_\gamma(Z|X_{\mathrm{obs}(M)})$ and $p_\theta(X_{\mathrm{obs}(M)}|Z)$ can be Gaussian, but a more relevant choice can be made depending on the situation and also on the type of variables (quantitative, categorical).

Finally, the neural network architecture must be chosen, along with associated parameters, notably:
* the number of hidden layers,
* the size (dimension) of the hidden layers in the neural networks.

In the following, for the latent variables, a standard Gaussian distribution will be chosen (this prior distribution is suitable most of the time) : $p(Z)=N(0_d,I_{d\times d})$. Pour $p_\gamma(Z|X_{\mathrm{obs}(M)})$, A Gaussian distribution with a mean and a diagonal covariance matrix is also chosen. For $p_\theta(X_{\mathrm{obs}(M)}|Z)$, a Student’s t-distribution will be used.

The neural networks for the encoder and the decoder share the same architecture, consisting of 3 layers. For example, for $p_\gamma(Z|X_{\mathrm{obs}(M)})$, we assume:
$f_\theta(Z)=\sigma(W_1\sigma(W_0 Z + b_0)+ b_1)$, where $\sigma$ is the ReLU activation function. The parameters of the Student’s t-distribution are then estimated with :
\begin{align*}
\mu_\theta&=W_\mu f_\theta(Z) + b_\mu \quad \textrm{(mean)} \\
\Sigma_\theta&=\textrm{Softplus}(\textrm{Diag}(W_\sigma f_\theta(Z) + b_\sigma))+10^{-3} \quad \textrm{(covariance matrix)} \\
\nu_\theta &= \textrm{Softplus}(W_\nu f_\theta(Z) + b_\nu)+3 \quad \textrm{(number of degrees of freedom)},
\end{align*}
We add $10^{-3}$ to effectively obtain a covariance matrix, and $3$ to have at least three degrees of freedom so that the tails are not too heavy. These choices are explained in more detail in the `MIWAE` method notebook mentioned above.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # determine si CUDA est disponible (utilisation GPU)

In [ ]:
h = 128 # number of hidden units in (same for all MLPs)
d_lat = 10 # dimension of the latent space
K = 20 # number of IS during training

In [ ]:
p_z = td.Independent(td.Normal(loc=torch.zeros(d_lat).to(device), scale=torch.ones(d_lat).to(device)), 1)

In [ ]:
decoder = nn.Sequential(
    torch.nn.Linear(d_lat, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, 3 * d),  # the decoder will output both the mean, the scale, and the number of degrees of freedoms (hence the 3*p)
)

In [ ]:
encoder = nn.Sequential(
    torch.nn.Linear(d, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, 2 * d_lat),  # the encoder will output both the mean and the diagonal covariance
)

In [ ]:
def weights_init(layer): # this function will be used for initializing the neural networks
  if type(layer) == nn.Linear: torch.nn.init.orthogonal_(layer.weight)

Below is the function that calculates the bound $L_K$ to be maximized.

In [ ]:
def miwae_loss(iota_x, mask):
  batch_size = iota_x.shape[0]
  out_encoder = encoder(iota_x)
  q_zgivenxobs = td.Independent(td.Normal(loc=out_encoder[..., :d_lat], scale=torch.nn.Softplus()(out_encoder[..., d_lat:(2*d_lat)])), 1)

  zgivenx = q_zgivenxobs.rsample([K])
  zgivenx_flat = zgivenx.reshape([K*batch_size, d_lat])

  out_decoder = decoder(zgivenx_flat)
  all_means_obs_model = out_decoder[..., :d]
  all_scales_obs_model = torch.nn.Softplus()(out_decoder[..., d:(2*d)]) + 0.001
  all_degfreedom_obs_model = torch.nn.Softplus()(out_decoder[..., (2*d):(3*d)]) + 3

  data_flat = torch.Tensor.repeat(iota_x,[K, 1]).reshape([-1, 1])
  tiledmask = torch.Tensor.repeat(mask,[K, 1])

  all_log_pxgivenz_flat = torch.distributions.StudentT(loc=all_means_obs_model.reshape([-1, 1]), scale=all_scales_obs_model.reshape([-1, 1]), df=all_degfreedom_obs_model.reshape([-1, 1])).log_prob(data_flat)
  all_log_pxgivenz = all_log_pxgivenz_flat.reshape([K * batch_size, d])

  logpxobsgivenz = torch.sum(all_log_pxgivenz * (1 - tiledmask), 1).reshape([K, batch_size])
  logpz = p_z.log_prob(zgivenx)
  logq = q_zgivenxobs.log_prob(zgivenx)

  neg_bound = -torch.mean(torch.logsumexp(logpxobsgivenz + logpz - logq,0))

  return neg_bound

## Question 2: standardization

Like many deep learning methods, data needs to be standardized.

The dataset containing missing values is standardized with the following code. What could be the problem here?

In [ ]:
mean_xmiss = np.nanmean(xmiss, 0)
std_xmiss = np.nanstd(xmiss, 0)
xmiss = (xmiss - mean_xmiss) / std_xmiss

ximp_0 = init_zero_imputation(xmiss, mask)

### Solution

The data are standardized by computing the mean and standard deviation on the zero-imputed data. These estimates may therefore be biased, but it can be assumed that this will not have an impact on the final model training.

## Question 3: model training

Which algorithm should be used to optimize the bound $L_K$ ?

### Solution

The mini-batch stochastic gradient descent (SGD) algorithm is used, recalled below in pseudo-code:

`for epoch=0 in NB_epochs:` # loop over the epochs
*   `Draw a random permutation of the data and create mini-batches of size n_batch`
*   `For each batch`: # loop over the batches
  + `Update the parameters`
$$ (\theta^{(t+1)},\gamma^{(t+1)})=(\theta^{(t)},\gamma^{(t)})-\lambda \: \partial_{(\theta,\gamma)} L_K(\theta^{(t)},\gamma^{(t)}), \quad \textrm{with $\lambda$ is the learning rate.}$$





The user selects the learning rate, the number of epochs, and the batch size.

In practice, the `Adam` optimization algorithm is preferred. It is an extension of `SGD` that uses an adaptive learning rate and momentum.

Below is the code for training the model.

In [ ]:
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=1e-3)

In [ ]:
encoder.to(device) # we'll use the GPU or CPU depending on the available device
decoder.to(device)

In [ ]:
miwae_loss_train = np.array([])
mse_train = np.array([])
mse_train2 = np.array([])
bs = 64 # batch size
n_epochs = 2002 # 1 epoch = all the data are used once
ximp_MIWAE = np.copy(ximp_0) # This will be out imputed data matrix

encoder.apply(weights_init)
decoder.apply(weights_init)

for ep in range(1, n_epochs):
  perm = np.random.permutation(n) # We use the "random reshuffling" version of SGD
  batches_data = np.array_split(ximp_0[perm,], n/bs)
  batches_mask = np.array_split(mask[perm,], n/bs)
  for it in range(len(batches_data)):
    optimizer.zero_grad()
    encoder.zero_grad()
    decoder.zero_grad()
    b_data = torch.from_numpy(batches_data[it]).float().to(device)
    b_mask = torch.from_numpy(batches_mask[it]).float().to(device)
    loss = miwae_loss(iota_x=b_data, mask=b_mask)
    loss.backward()
    optimizer.step()
  if ep % 100 == 1:
    print('Epoch %g' %ep)
    print('MIWAE likelihood bound  %g' %(-np.log(K) - miwae_loss(iota_x=torch.from_numpy(ximp_0).float().to(device), mask=torch.from_numpy(mask).float().to(device)).cpu().data.numpy())) # Gradient step

## Question 4: imputation

The final step consists in predicting the missing values. A simple imputation will be performed, but multiple imputation is also possible (see [specific notebook](https://github.com/pamattei/miwae/blob/master/Tensorflow%202%20notebooks/MIWAE_UCI_single_multiple-imputation.ipynb)).

Which quantity should be computed to predict the missing values?

### Solution

The idea of `MIWAE` is to compute the conditional expectation of $X_{\mathrm{mis}(M)}|X_{\mathrm{obs}(M)}$ as follows:

   
   \begin{align*}
        &\mathbb{E}[X_{\mathrm{mis}(M)}|X_{\mathrm{obs}(M)}] \\
        &=\int X_{\mathrm{mis}(M)} p_\theta(X_{\mathrm{mis}(M)}|X_{\mathrm{obs}(M)})dX_{\mathrm{mis}(M)} \\
        &= \int \int X_{\mathrm{mis}(M)} p_\theta(X_{\mathrm{mis}(M)}|X_{\mathrm{obs}(M)},Z)p_\theta(Z|X_{\mathrm{obs}(M)})dZdX_{\mathrm{mis}(M)} \\
        &= \int \int X_{\mathrm{mis}(M)} \frac{p_\theta(Z|X_{\mathrm{obs}(M)})}{{p_\gamma(Z|X_{\mathrm{obs}(M)})}}{p_\theta(X_{\mathrm{mis}(M)}|X_{\mathrm{obs}(M)},Z)p_\gamma(Z|X_{\mathrm{obs}(M)})}dZdX_{\mathrm{mis}(M)}
    \end{align*}
It is not explicit, so a self-normalized importance sampling method is used (more details on this sampling method are available [here, Section 9.2](https://artowen.su.domains/mc/)):
$$\sum_{l=1}^L w_l X_{\mathrm{mis}(M)}^{(l)}$$
with $w_l=\frac{r_l}{\sum_{l=1}^L r_l}$, $r_l=\frac{p_\theta(X_{\mathrm{obs}(M)}|Z^{(l)})p(Z^{(l)})}{{p_\gamma(Z^{(l)}|X_{\mathrm{obs}(M)})}}$, et
$(X_{\mathrm{mis}(M)}^{(l)},Z^{(l)}) \overset{\mathrm{iid}}{\sim} {p_\theta(X_{\mathrm{mis}(M)}|X_{\mathrm{obs}(M)},Z)p_\gamma(Z|X_{\mathrm{obs}(M)})}$

In [ ]:
def miwae_impute(iota_x,mask, L):
  batch_size = iota_x.shape[0]
  out_encoder = encoder(iota_x)
  q_zgivenxobs = td.Independent(td.Normal(loc=out_encoder[..., :d_lat], scale=torch.nn.Softplus()(out_encoder[..., d_lat:(2*d_lat)])), 1)

  zgivenx = q_zgivenxobs.rsample([L])
  zgivenx_flat = zgivenx.reshape([L * batch_size, d_lat])

  out_decoder = decoder(zgivenx_flat)
  all_means_obs_model = out_decoder[..., :d]
  all_scales_obs_model = torch.nn.Softplus()(out_decoder[..., d:(2*d)]) + 0.001
  all_degfreedom_obs_model = torch.nn.Softplus()(out_decoder[..., (2*d):(3*d)]) + 3

  data_flat = torch.Tensor.repeat(iota_x,[L, 1]).reshape([-1, 1]).to(device)
  tiledmask = torch.Tensor.repeat(mask,[L, 1]).to(device)

  all_log_pxgivenz_flat = torch.distributions.StudentT(loc=all_means_obs_model.reshape([-1, 1]), scale=all_scales_obs_model.reshape([-1, 1]), df=all_degfreedom_obs_model.reshape([-1, 1])).log_prob(data_flat)
  all_log_pxgivenz = all_log_pxgivenz_flat.reshape([L * batch_size, d])

  logpxobsgivenz = torch.sum(all_log_pxgivenz * (1 - tiledmask), 1).reshape([L, batch_size])
  logpz = p_z.log_prob(zgivenx)
  logq = q_zgivenxobs.log_prob(zgivenx)

  xgivenz = td.Independent(td.StudentT(loc=all_means_obs_model, scale=all_scales_obs_model, df=all_degfreedom_obs_model), 1)

  imp_weights = torch.nn.functional.softmax(logpxobsgivenz + logpz - logq, 0) # these are w_1,....,w_L for all observations in the batch
  xms = xgivenz.sample().reshape([L, batch_size, d])
  xm = torch.einsum('ki,kij->ij', imp_weights, xms)

  return xm


In [ ]:
ximp_MIWAE[mask] = miwae_impute(iota_x=torch.from_numpy(ximp_0).float().to(device), mask=torch.from_numpy(mask).float().to(device), L=10).cpu().data.numpy()[mask]
ximp_MIWAE_destandardized = ximp_MIWAE * std_xmiss + mean_xmiss
mse_MIWAE = mse(ximp_MIWAE_destandardized, xfull)
print("MSE MIWAE:", mse_MIWAE)